<a href="https://colab.research.google.com/github/hananbahtiti/Hybrid-Intrusion-detection-Systems/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#test

In [ ]:
!wget -O NGIDS-DS.rar https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download

--2025-04-22 08:38:40--  https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download
Resolving unsworks.unsw.edu.au (unsworks.unsw.edu.au)... 54.253.215.118
Connecting to unsworks.unsw.edu.au (unsworks.unsw.edu.au)|54.253.215.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content [following]
--2025-04-22 08:38:42--  https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content
Reusing existing connection to unsworks.unsw.edu.au:443.
HTTP request sent, awaiting response... 200 200
Length: unspecified [application/x-rar-compressed]
Saving to: ‘NGIDS-DS.rar’

NGIDS-DS.rar            [               <=>  ] 941.51M  14.0MB/s    in 70s     

2025-04-22 08:39:52 (13.4 MB/s) - ‘NGIDS-DS.rar’ saved [987249484]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unrar x /content/NGIDS-DS.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/NGIDS-DS.rar

Creating    NGIDS                                                     OK
Creating    NGIDS/NGIDS-DS-v1                                         OK
Extracting  NGIDS/NGIDS-DS-v1/feature_descr.csv                            0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/ground_truth.csv                             0%  OK 
Creating    NGIDS/NGIDS-DS-v1/host logs                               OK
Extracting  NGIDS/NGIDS-DS-v1/host logs/1.csv                              0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/10.csv                             0%  1%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/11.csv                             1%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/12.csv                             1%  2%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/13.csv                             2%  OK 
Extrac

In [ ]:
!rm -rf /content/NGIDS

#class

In [ ]:
!pip install tqdm

In [42]:
import pandas as pd
import numpy as np
import os
import glob
import math
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [ ]:
class HostPreprocessing():
  def __init__(self):
    pass

  def __file_path_collection(self, folder_path):
    try:
      joined_list = []
      for file_name in os.listdir(folder_path):
        file_name = os.path.join(folder_path, file_name)
        if os.path.isfile(file_name) and file_name.endswith('.csv'):
          joined_list.append(file_name)
      return joined_list

    except Exception as e:
      return f"An error occurred: {e}"


  def rename_columns(self, folder_path):
    files = self.__file_path_collection(folder_path)
    try:
      for file in tqdm(files, desc="Renaming columns") :
        df = pd.read_csv(file)
        column_name = df.columns.tolist()
        df.rename(columns={
            column_name[0]:'Date',
            column_name[1]:'Time',
            column_name[2]:'Unique_Identification',
            column_name[3]:'Execution_Path',
            column_name[4]:'System_Calls_Identifiers',
            column_name[5]:"Event's_Unique_Identification",
            column_name[6]:'attacks',
            column_name[7]:"Sub_Type_Attack",
            column_name[8]:"Label"
            }, inplace=True)
        df.to_csv(file, index=False)
      return f"done files..."

    except Exception as e:
      return f"An error occurred: {e}"


  def data_encoding(self, folder_path, column_number : int):
    files = self.__file_path_collection(folder_path)
    encoder = LabelEncoder()
    all_value = []
    try:
      for file in tqdm(files, desc="Extract all values from files"):
        df = pd.read_csv(file)
        all_value.extend(df.iloc[:, column_number].dropna().unique())
        #print( len(df.iloc[:, column_number].dropna().unique()) , file )
      encoder.fit(all_value)

      for file in tqdm(files, desc="encoder data"):
        df = pd.read_csv(file)
        df.iloc[:, column_number] = encoder.transform(df.iloc[:, column_number])
        #print(len(df.iloc[:, column_number].dropna().unique()), df.iloc[:, column_number].dropna().unique(), file )
        df.to_csv(file, index=False)
      return f"done file..."

    except Exception as e:
      return f"An error occurred: {e}"


  def paths_encoding(self, folder_path, column_number: int):
    files = self.__file_path_collection(folder_path)
    all_unique_value = []
    unique_value = []
    try:
      for file in tqdm(files, desc="read all files"):
        df = pd.read_csv(file)
        unique_value.extend(df.iloc[:,column_number].unique())
        #df["Tokenized"] = unique_value.apply(lambda p: p.strip("/").split("/"))
        for tokens in unique_value:
          all_unique_value.extend(tokens)

      print( len(all_unique_value), all_unique_value)

    except Exception as e:
      return f"An error occurred: {e}"


  def file_collection(self, folder_path):
    try:
      file_paths = self.__file_path_collection(folder_path)
      if file_paths:
        output_folder = os.path.join(os.getcwd(),'files')
        os.makedirs(output_folder ,exist_ok=True)
        output_file = os.path.join(output_folder,"final.csv")

        for file in tqdm(file_paths, desc="read all files"):

          first = True
          chunk = pd.read_csv(file, chunksize=10000)
          for part in chunk:
            part.to_csv(output_file, mode='a', header=first, index=False)
            first = False

      return  f"done file..."

    except Exception as e:
      return f"An error occurred: {e}"


a = HostPreprocessing()
#a.file_path_collection("/content/NGIDS/NGIDS-DS-v1/host logs")
a.file_collection("/content/NGIDS/NGIDS-DS-v1/host logs")
#a.rename_columns( '/content/NGIDS/NGIDS-DS-v1/host logs')
#a.data_encoding('/content/NGIDS/NGIDS-DS-v1/host logs', column_number=6)
#a.data_encoding('/content/NGIDS/NGIDS-DS-v1/host logs', column_number=7)

#a.paths_encoding('/content/NGIDS/NGIDS-DS-v1/host logs', column_number=3)

read all files:   2%|▏         | 2/99 [00:06<05:59,  3.71s/it]

In [ ]:
import pandas as pd

# reading two csv files
data1 = pd.read_csv('/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv')
data2 = pd.read_csv('datasets/borrower.csv')

# using merge function by setting how='inner'
output1 = pd.merge(data1, data2,
                   on='LOAN_NO',
                   how='inner')

# displaying result
print(output1)

#network




In [ ]:
pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.3 MB/s eta 0:00:00


In [14]:
import csv
import json
import os
import re
from tqdm import tqdm
from scapy.all import PcapReader

In [ ]:
class NetworkPreprocessing:
  def __init__(self):
    pass


  def extract_tcp_option(self, option, key):
    for opt in option:
      if opt[0] == key:
        return opt[1]
    return None


  def delete_data(self, file_path):
    df = pd.read_csv(file_path)
    df = df.loc[:, df.nunique() > 1]
    df.to_csv(file_path, index=False)
    return f"Columns containing only one value are deleted..."

  def convert_pcap_csv(self, pcap_file, csv_file):
    try:
      print("Countig packages...")
      with PcapReader(pcap_file) as count_reader:
        total_packets = sum(1 for _ in count_reader)
        print(f"Total number of packages: {total_packets}")

      print("Start converting to csv file...")
      with PcapReader(pcap_file) as packets:
        with open(csv_file, mode='w', newline="") as file:
          writer = csv.writer(file)
          writer.writerow([
              "Packet Number", "Ethernet DST", "Ethernet SRC", "Ethernet Type",
              "IP Version", "IP IHL", "IP TOS", "IP Length", "IP ID", "IP Flags",
              "IP Fragmentation", "IP TTL", "IP Proto", "IP Checksum",
              "IP Src", "IP Dst", "TCP Src Port", "TCP Dst Port", "TCP Seq",
              "TCP Ack", "TCP Data Offset", "TCP Reserved", "TCP Flags",
              "TCP Window", "TCP Checksum", "TCP Urgent Pointer", "TCP Options MSS",
              "TCP Options WScale", "TCP Options NOP","Hexdump"
          ])

          for i, packet in enumerate(tqdm(packets,total=total_packets, desc="Packet processing")):
            if packet.haslayer('Ethernet'):
              eth_dst = packet['Ethernet'].dst
              eth_src = packet['Ethernet'].src
              eth_type = packet['Ethernet'].type
            else:
              eth_dst = eth_src = eth_type = None

            if packet.haslayer('IP'):
              ip_version = packet['IP'].version
              ip_ihl = packet['IP'].ihl
              ip_tos = packet['IP'].tos
              ip_len = packet['IP'].len
              ip_id = packet['IP'].id
              ip_flags = packet['IP'].flags
              ip_frag = packet['IP'].frag
              ip_ttl = packet['IP'].ttl
              ip_proto = packet['IP'].proto
              ip_chksum = packet['IP'].chksum
              ip_src = packet['IP'].src
              ip_dst = packet['IP'].dst
            else:
              ip_version = ip_ihl = ip_tos = ip_len = ip_id = ip_flags = ip_frag = ip_ttl = ip_proto = ip_chksum = ip_src = ip_dst = None

            if packet.haslayer('TCP'):
              tcp_sport = packet['TCP'].sport
              tcp_dport = packet['TCP'].dport
              tcp_seq = packet['TCP'].seq
              tcp_ack = packet['TCP'].ack
              tcp_dataofs = packet['TCP'].dataofs
              tcp_reserved = packet['TCP'].reserved
              tcp_flags = packet['TCP'].flags
              tcp_window = packet['TCP'].window
              tcp_chksum = packet['TCP'].chksum
              tcp_urgptr = packet['TCP'].urgptr
              tcp_options = packet['TCP'].options

              tcp_mss = self.extract_tcp_option(tcp_options, 'MSS')
              tcp_wscale = self.extract_tcp_option(tcp_options, 'WScale')
              tcp_nop_count = self.extract_tcp_option(tcp_options, 'NOP')

            else:
              tcp_sport = tcp_dport = tcp_seq = tcp_ack = tcp_dataofs = tcp_reserved = tcp_flags = tcp_window = tcp_chksum = tcp_urgptr = tcp_mss = tcp_wscale = tcp_nop_count = None

            raw_data = packet.original.hex()
            writer.writerow([
                i + 1, eth_dst, eth_src, eth_type,
                  ip_version, ip_ihl, ip_tos, ip_len, ip_id, ip_flags,
                  ip_frag, ip_ttl, ip_proto, ip_chksum,
                  ip_src, ip_dst, tcp_sport, tcp_dport, tcp_seq,
                  tcp_ack, tcp_dataofs, tcp_reserved, tcp_flags,
                  tcp_window, tcp_chksum, tcp_urgptr, tcp_mss, tcp_wscale, tcp_nop_count,
                  raw_data
            ])

      return f"The file has been converted to csv successfully."
    except Exception as e:
      return f"An error occurred: {e}"

a = NetworkPreprocessing()
#a.convert_pcap_csv(pcap_file='/content/NGIDS/NGIDS-DS-v1/NGIDS.pcap',
#                   csv_file='/content/drive/MyDrive/hybrid_IDS/network.csv')
#a.delete_data(file_path='/content/drive/MyDrive/hybrid_IDS/network.csv')

'Columns containing only one value are deleted...'

In [39]:
class PcapFileProcessing:
  def __init__(self) -> None:
    pass



  def delete_data(self, file_path):
    df = pd.read_csv(file_path)
    df = df.loc[:, df.nunique() > 1]
    df.to_csv(file_path, index=False)
    return f"Columns containing only one value are deleted..."


  def date_time_columns(self, file_path, csv_file):
    try:
        import pandas as pd

        # قراءة الملف مع إجبار كل الأعمدة كنص
        df = pd.read_csv(file_path, dtype=str)

        # إزالة الفراغات من أسماء الأعمدة
        df.columns = df.columns.str.strip()

        # التأكد من وجود العمود
        if 'timestamp' not in df.columns:
            return "خطأ: العمود 'timestamp' غير موجود في الملف."

        # تنظيف القيم النصية
        df['timestamp'] = df['timestamp'].astype(str).str.strip()

        # تحويل التاريخ
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', dayfirst=True)

        if df['timestamp'].isnull().all():
            return "كل القيم في 'timestamp' غير صالحة ولم يتم تحويلها."

        # استخراج date و time
        df['date'] = df['timestamp'].dt.strftime('%d/%m/%Y')
        df['time'] = df['timestamp'].dt.strftime('%H:%M:%S')

        # حفظ النتيجة
        df.to_csv(csv_file, index=False)

        return "تم تحويل timestamp بنجاح إلى date و time."

    except Exception as e:
        return f"حدث خطأ أثناء التحويل: {e}"







  def convert_log_csv(self, log_file, csv_file):
    try:
      print("Start converting to csv file...")
      with open(log_file, "r") as logfile:
        log_data = logfile.read()
      pattern = re.compile(
          r'(?P<timestamp>\d{2}/\d{2}/\d{4}-\d{2}:\d{2}:\d{2}\.\d+)\s+\[\*\*\]\s+\[(?P<sid>[^]]+)\]\s+(?P<signature>.*?)\s+\[\*\*\]\s+\[Classification:\s+(?P<classification>.*?)\]\s+\[Priority:\s+(?P<priority>\d+)\]\s+\{(?P<protocol>\w+)\}\s+(?P<src_ip>\d+\.\d+\.\d+\.\d+):(?P<src_port>\d+)\s+->\s+(?P<dst_ip>\d+\.\d+\.\d+\.\d+):(?P<dst_port>\d+)'
          )
      with open(csv_file, "w", newline="") as csvfile:
        fieldnames = ['timestamp', 'sid', 'signature', 'classification',
                      'priority', 'protocol','src_ip', 'src_port', 'dst_ip', 'dst_port']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for match in pattern.finditer(log_data):
          writer.writerow(match.groupdict())

      return f"The file has been converted to csv successfully."

    except Exception as e:
      return f"An error occurred: {e}"


  def convert_json_csv(self, json_file, csv_file):
    try:
      print("Start converting JSON to CSV...")
      with open(json_file, 'r') as f:
        data = [json.loads(line) for line in f if line.strip()]

      fieldnames = set()
      for row in data:
          fieldnames.update(row.keys())

      fieldnames = list(fieldnames)

      with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(fieldnames)

        for row in data:
          writer.writerow(row.get(field, '') for field in fieldnames)

      return f"The file has been converted to csv successfully."

    except Exception as e:
      return f"An error occurred: {e}"



a = PcapFileProcessing()
#a.convert_log_csv( log_file="/content/drive/MyDrive/hybrid_IDS/fast.log",
#                  csv_file="/content/drive/MyDrive/hybrid_IDS/alerts.csv")

#a.convert_json_csv(json_file='/content/drive/MyDrive/hybrid_IDS/eve.json',
#                   csv_file='/content/drive/MyDrive/hybrid_IDS/eve.csv')

#a.delete_data( file_path='/content/drive/MyDrive/hybrid_IDS/eve.csv')
a.date_time_columns(file_path='/content/drive/MyDrive/hybrid_IDS/eve.csv',
                    csv_file='/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv')

'تم تحويل timestamp بنجاح إلى date و time.'

In [58]:
import pandas as pd
import os
from tqdm import tqdm

# المسار إلى مجلد ملفات host
host_folder = '/content/NGIDS/NGIDS-DS-v1/host logs'

# ملف الشبكة الرئيسي
network_df = pd.read_csv('/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv')

# تأكد من تنسيق الأعمدة المشتركة في network
network_df['Date'] = network_df['Date'].astype(str).str.strip()
network_df['Time'] = network_df['Time'].astype(str).str.strip()

# الحصول على قائمة ملفات host
host_files = [f for f in os.listdir(host_folder) if f.endswith('.csv')]

# DataFrame لتجميع النتائج
all_merged = []

for host_file in tqdm(host_files):
    host_path = os.path.join(host_folder, host_file)
    host_df = pd.read_csv(host_path)

    # تنسيق الأعمدة المشتركة
    host_df['Date'] = host_df['Date'].astype(str).str.strip()
    host_df['Time'] = host_df['Time'].astype(str).str.strip()

    # دمج داخلي
    merged_df = pd.merge(host_df, network_df, on=['Date', 'Time'], how='inner')

    # إضافة النتائج المدموجة للقائمة
    all_merged.append(merged_df)

# دمج كل النتائج في ملف واحد
final_df = pd.concat(all_merged, ignore_index=True)

# حفظ الملف الموحد
output_path = '/content/drive/MyDrive/hybrid_IDS/dataset/all/final_merged.csv'
final_df.to_csv(output_path, index=False)

print(f"تم دمج جميع الملفات المطابقة وحفظها في: {output_path} | عدد الصفوف: {len(final_df)}")

<ipython-input-58-981dbd2e52f2>:9: DtypeWarning: Columns (11,16,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  network_df = pd.read_csv('/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv')
100%|██████████| 99/99 [03:31<00:00,  2.14s/it]

تم دمج جميع الملفات المطابقة وحفظها في: /content/drive/MyDrive/hybrid_IDS/dataset/all/final_merged.csv | عدد الصفوف: 0


In [48]:
df = pd.read_csv("/content/NGIDS/NGIDS-DS-v1/host logs/1.csv")
df.info()
#df.columns
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column                         Non-Null Count    Dtype 
---  ------                         --------------    ----- 
 0   Date                           1000000 non-null  object
 1   Time                           1000000 non-null  object
 2   Unique_Identification          1000000 non-null  int64 
 3   Execution_Path                 1000000 non-null  object
 4   System_Calls_Identifiers       1000000 non-null  int64 
 5   Event's_Unique_Identification  1000000 non-null  int64 
 6   attacks                        1000000 non-null  object
 7   Sub_Type_Attack                1000000 non-null  object
 8   Label                          1000000 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 68.7+ MB


,Date,Time,Unique_Identification,Execution_Path,System_Calls_Identifiers,Event's_Unique_Identification,attacks,Sub_Type_Attack,Label
0,11/03/2016,2:45:06,1804,/bin/dbus-daemon,256,45352,normal,normal,0
1,11/03/2016,2:45:06,2133,/usr/lib/i386-linux-gnu/gconf/gconfd-2,168,45372,normal,normal,0
2,11/03/2016,2:45:35,4528,/usr/bin/python3.4,3,39459,normal,normal,0
3,11/03/2016,2:45:44,1847,/usr/bin/ibus-daemon,102,37263,normal,normal,0
4,11/03/2016,2:45:44,1907,/usr/lib/ibus/ibus-ui-gtk3,168,37896,normal,normal,0


In [54]:
df = pd.read_csv("/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv")
df.rename(columns={'date':'Date', 'time':'Time'}, inplace=True)
df.to_csv('/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv', index=False)
df.info()
#df.columns
df.head()

<ipython-input-54-1b2cd2840b38>:1: DtypeWarning: Columns (11,16,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286150 entries, 0 to 286149
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ike         100 non-null     object 
 1   dns         1000 non-null    object 
 2   sip         300 non-null     object 
 3   flow        284361 non-null  object 
 4   dest_ip     286148 non-null  object 
 5   src_port    285848 non-null  float64
 6   dest_port   285848 non-null  float64
 7   proto       286148 non-null  object 
 8   app_proto   8916 non-null    object 
 9   pcap_cnt    6787 non-null    float64
 10  src_ip      286148 non-null  object 
 11  timestamp   137399 non-null  object 
 12  alert       5100 non-null    object 
 13  krb5        94 non-null      object 
 14  event_type  286150 non-null  object 
 15  snmp        93 non-null      object 
 16  stats       2 non-null       object 
 17  tcp         270253 non-null  object 
 18  flow_id     286048 non-null  float64
 19  Da

,ike,dns,sip,flow,dest_ip,src_port,dest_port,proto,app_proto,pcap_cnt,...,timestamp,alert,krb5,event_type,snmp,stats,tcp,flow_id,Date,Time
0,NaN,NaN,NaN,NaN,10.40.85.32,0.0,0.0,TCP,NaN,500.0,...,2016-10-03 17:49:10.892710+02:00,"{'action': 'allowed', 'gid': 1, 'signature_id'...",NaN,alert,NaN,NaN,NaN,NaN,03/10/2016,17:49:10
1,NaN,NaN,NaN,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte...",10.40.85.32,1024.0,4096.0,TCP,NaN,2260.0,...,2016-10-03 17:49:44.321710+02:00,"{'action': 'allowed', 'gid': 1, 'signature_id'...",NaN,alert,NaN,NaN,NaN,2.558362e+14,03/10/2016,17:49:44
2,NaN,NaN,NaN,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte...",10.40.85.32,1024.0,4096.0,TCP,NaN,2260.0,...,2016-10-03 17:49:44.321710+02:00,"{'action': 'allowed', 'gid': 1, 'signature_id'...",NaN,alert,NaN,NaN,NaN,2.558362e+14,03/10/2016,17:49:44
3,NaN,NaN,NaN,NaN,10.40.85.32,43575.0,69.0,UDP,NaN,2232.0,...,2016-10-03 17:49:36.996126+02:00,NaN,NaN,tftp,NaN,NaN,NaN,5.620575e+13,03/10/2016,17:49:36
4,NaN,NaN,NaN,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte...",10.40.85.31,52547.0,1280.0,TCP,NaN,NaN,...,2016-10-03 17:48:59.810052+02:00,NaN,NaN,flow,NaN,NaN,"{'tcp_flags': '00', 'tcp_flags_ts': '00', 'tcp...",8.582737e+14,03/10/2016,17:48:59


In [52]:
p = pd.read_csv("/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv")
p.info()

<ipython-input-52-cb9b34261285>:1: DtypeWarning: Columns (11,16,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  p = pd.read_csv("/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286150 entries, 0 to 286149
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ike         100 non-null     object 
 1   dns         1000 non-null    object 
 2   sip         300 non-null     object 
 3   flow        284361 non-null  object 
 4   dest_ip     286148 non-null  object 
 5   src_port    285848 non-null  float64
 6   dest_port   285848 non-null  float64
 7   proto       286148 non-null  object 
 8   app_proto   8916 non-null    object 
 9   pcap_cnt    6787 non-null    float64
 10  src_ip      286148 non-null  object 
 11  timestamp   137399 non-null  object 
 12  alert       5100 non-null    object 
 13  krb5        94 non-null      object 
 14  event_type  286150 non-null  object 
 15  snmp        93 non-null      object 
 16  stats       2 non-null       object 
 17  tcp         270253 non-null  object 
 18  flow_id     286048 non-null  float64
 19  da

In [40]:
df = pd.read_csv("/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv")
#df['Sub_Type_Attack'].value_counts()
df.info()
df.head()

<ipython-input-40-f8f0e81e485b>:1: DtypeWarning: Columns (11,16,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286150 entries, 0 to 286149
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ike         100 non-null     object 
 1   dns         1000 non-null    object 
 2   sip         300 non-null     object 
 3   flow        284361 non-null  object 
 4   dest_ip     286148 non-null  object 
 5   src_port    285848 non-null  float64
 6   dest_port   285848 non-null  float64
 7   proto       286148 non-null  object 
 8   app_proto   8916 non-null    object 
 9   pcap_cnt    6787 non-null    float64
 10  src_ip      286148 non-null  object 
 11  timestamp   137399 non-null  object 
 12  alert       5100 non-null    object 
 13  krb5        94 non-null      object 
 14  event_type  286150 non-null  object 
 15  snmp        93 non-null      object 
 16  stats       2 non-null       object 
 17  tcp         270253 non-null  object 
 18  flow_id     286048 non-null  float64
 19  da

,ike,dns,sip,flow,dest_ip,src_port,dest_port,proto,app_proto,pcap_cnt,...,timestamp,alert,krb5,event_type,snmp,stats,tcp,flow_id,date,time
0,NaN,NaN,NaN,NaN,10.40.85.32,0.0,0.0,TCP,NaN,500.0,...,2016-10-03 17:49:10.892710+02:00,"{'action': 'allowed', 'gid': 1, 'signature_id'...",NaN,alert,NaN,NaN,NaN,NaN,03/10/2016,17:49:10
1,NaN,NaN,NaN,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte...",10.40.85.32,1024.0,4096.0,TCP,NaN,2260.0,...,2016-10-03 17:49:44.321710+02:00,"{'action': 'allowed', 'gid': 1, 'signature_id'...",NaN,alert,NaN,NaN,NaN,2.558362e+14,03/10/2016,17:49:44
2,NaN,NaN,NaN,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte...",10.40.85.32,1024.0,4096.0,TCP,NaN,2260.0,...,2016-10-03 17:49:44.321710+02:00,"{'action': 'allowed', 'gid': 1, 'signature_id'...",NaN,alert,NaN,NaN,NaN,2.558362e+14,03/10/2016,17:49:44
3,NaN,NaN,NaN,NaN,10.40.85.32,43575.0,69.0,UDP,NaN,2232.0,...,2016-10-03 17:49:36.996126+02:00,NaN,NaN,tftp,NaN,NaN,NaN,5.620575e+13,03/10/2016,17:49:36
4,NaN,NaN,NaN,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte...",10.40.85.31,52547.0,1280.0,TCP,NaN,NaN,...,2016-10-03 17:48:59.810052+02:00,NaN,NaN,flow,NaN,NaN,"{'tcp_flags': '00', 'tcp_flags_ts': '00', 'tcp...",8.582737e+14,03/10/2016,17:48:59


In [23]:
df.nunique()

,0
ike,6
dns,802
sip,300
flow,284161
dest_ip,8
src_port,63641
dest_port,11523
proto,5
app_proto,8
pcap_cnt,6394


In [20]:
df.dropna(inplace=True)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ike         0 non-null      object 
 1   dns         0 non-null      object 
 2   sip         0 non-null      object 
 3   flow        0 non-null      object 
 4   dest_ip     0 non-null      object 
 5   src_port    0 non-null      float64
 6   dest_port   0 non-null      float64
 7   proto       0 non-null      object 
 8   app_proto   0 non-null      object 
 9   pcap_cnt    0 non-null      float64
 10  src_ip      0 non-null      object 
 11  timestamp   0 non-null      object 
 12  alert       0 non-null      object 
 13  krb5        0 non-null      object 
 14  event_type  0 non-null      object 
 15  snmp        0 non-null      object 
 16  stats       0 non-null      object 
 17  tcp         0 non-null      object 
 18  flow_id     0 non-null      float64
dtypes: float64(4), object(15)
memory usage: 0.0+ b

,ike,dns,sip,flow,dest_ip,src_port,dest_port,proto,app_proto,pcap_cnt,src_ip,timestamp,alert,krb5,event_type,snmp,stats,tcp,flow_id


#test

In [ ]:
unique_attacks = list(set([item for sub in c for item in sub]))
unique_attacks

['Exploits',
 'Generic',
 'Shellcode',
 'Worms',
 'Reconnaissance',
 'Denial of Service',
 'Backdoors',
 'normal']

In [ ]:
pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.6 MB/s eta 0:00:00


In [ ]:
df['TCP Options'].value_counts()

,count
TCP Flags,
S,799771
R,264798
RA,961
FA,100


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/hybrid_IDS/network.csv")
#df['Sub_Type_Attack'].value_counts()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094232 entries, 0 to 1094231
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Packet Number     1094232 non-null  int64  
 1   Ethernet DST      1094232 non-null  object 
 2   Ethernet SRC      1094232 non-null  object 
 3   Ethernet Type     1094232 non-null  int64  
 4   IP IHL            1079094 non-null  float64
 5   IP Length         1079094 non-null  float64
 6   IP ID             1079094 non-null  float64
 7   IP Fragmentation  1079094 non-null  float64
 8   IP TTL            1079094 non-null  float64
 9   IP Proto          1079094 non-null  float64
 10  IP Checksum       1079094 non-null  float64
 11  IP Src            1079094 non-null  object 
 12  IP Dst            1079094 non-null  object 
 13  TCP Src Port      1065930 non-null  float64
 14  TCP Dst Port      1065930 non-null  float64
 15  TCP Seq           1065930 non-null  float64
 16  

,Packet Number,Ethernet DST,Ethernet SRC,Ethernet Type,IP IHL,IP Length,IP ID,IP Fragmentation,IP TTL,IP Proto,...,TCP Src Port,TCP Dst Port,TCP Seq,TCP Ack,TCP Data Offset,TCP Flags,TCP Window,TCP Checksum,TCP Options MSS,Hexdump
0,1,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21299.0,0.0,32.0,6.0,...,60294.0,80.0,4.048308e+09,0.0,9.0,S,5792.0,16020.0,1460.0,021ac5000000021ac50000000800450000385333400020...
1,2,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21281.0,0.0,32.0,6.0,...,22311.0,1024.0,4.048306e+09,0.0,9.0,S,5792.0,54925.0,1460.0,021ac5000000021ac50000000800450000385321400020...
2,3,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21467.0,0.0,32.0,6.0,...,63239.0,27322.0,4.100437e+09,0.0,9.0,S,5792.0,38924.0,1460.0,021ac5000000021ac500000008004500003853db400020...
3,4,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21295.0,0.0,32.0,6.0,...,37241.0,23962.0,4.160434e+09,0.0,9.0,S,5792.0,21918.0,1460.0,021ac5000000021ac5000000080045000038532f400020...
4,5,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21469.0,0.0,32.0,6.0,...,17808.0,80.0,2.070854e+09,0.0,9.0,S,5792.0,48977.0,1460.0,021ac5000000021ac500000008004500003853dd400020...
